# Sistema de Simulación Logística - Tests y Validaciones

**Proyecto:** Sistema de Gestión Logística LIA S.A.C.

Este notebook documenta todos los tests del sistema con sus resultados.

---

## Test 1: `test_modules.py`

**Qué hace:** Verifica que todos los módulos del sistema (`catalogos`, `demanda`, `inventario`, `picking`, `transporte`, `indicadores`, `alertas`) se importen correctamente y que las clases principales (`GestionInventario`, `GestionTransporte`) se puedan instanciar sin errores.

**Comando de ejecución:**
```bash
.venv\Scripts\python.exe tests\test_modules.py
```

### Resultado del Test 1:

*[Pegar aquí captura de pantalla]*

---

## Test 2: `test_stock_logic.py`

**Qué hace:** Valida la lógica de stock comprometido del sistema. Prueba dos casos:
1. **Pedido Normal:** Verifica que el stock se compromete y libera correctamente cuando hay suficiente inventario
2. **Pedido Excesivo (Quiebre):** Verifica que cuando la demanda supera el stock disponible, solo se compromete y despacha lo disponible, sin generar stock negativo

**Comando de ejecución:**
```bash
.venv\Scripts\python.exe tests\test_stock_logic.py
```

### Resultado del Test 2:

*[Pegar aquí captura de pantalla]*

---

## Test 3: `test_compras_kardex.py`

**Qué hace:** Ejecuta una simulación de 15 días y verifica:
- Que el sistema genera órdenes de compra automáticamente cuando el stock llega al punto de reorden
- Que las órdenes se marcan como "Recibidas" después del lead time
- Que todos los movimientos de recepción (COMPRA_RECEPCION) se registran correctamente en el Kardex

**Comando de ejecución:**
```bash
.venv\Scripts\python.exe tests\test_compras_kardex.py
```

### Resultado del Test 3:

*[Pegar aquí captura de pantalla]*

---

## Test 4: `verify_changes.py`

**Qué hace:** Ejecuta una verificación integral del sistema con 4 validaciones:
1. **Stock No Negativo:** Verifica que el stock físico nunca sea negativo
2. **Registro de Stockouts:** Valida que el sistema registre backlog y ventas perdidas
3. **Capacidad de Transporte:** Verifica que ningún despacho exceda la capacidad del vehículo
4. **KPIs Válidos:** Confirma que OTIF y Fill Rate estén en rango 0-100%

**Comando de ejecución:**
```bash
.venv\Scripts\python.exe tests\verify_changes.py
```

### Resultado del Test 4:

*[Pegar aquí captura de pantalla]*

---

## Test 5: `debug_chart_data.py`

**Qué hace:** Ejecuta una simulación de 7 días y valida la estructura de datos para gráficos:
- Verifica que `df_productos` tenga la estructura correcta
- Valida que los snapshots diarios de inventario sean consistentes
- Calcula el valor total del inventario día a día
- Verifica que todos los SKUs estén presentes en los DataFrames

**Comando de ejecución:**
```bash
.venv\Scripts\python.exe tests\debug_chart_data.py
```

### Resultado del Test 5:

*[Pegar aquí captura de pantalla]*

---

## Test 6: `verify_corrections.py`

**Qué hace:** Ejecuta una simulación de 5 días y valida funcionalidades clave del sistema:
1. **KPIs Diarios:** Verifica el cálculo correcto de OTIF, Fill Rate y Backlog Rate por día
2. **KPIs Globales:** Valida los promedios globales y métricas consolidadas
3. **Estructura de Pedidos:** Verifica que los pedidos tengan las columnas de cantidad correctas
4. **Kardex Completo:** Valida que todos los movimientos estén registrados por tipo
5. **Referencias:** Confirma que las referencias (ID_Pedido, ID_Compra) estén presentes en el Kardex

**Comando de ejecución:**
```bash
.venv\Scripts\python.exe tests\verify_corrections.py
```

### Resultado del Test 6:

*[Pegar aquí captura de pantalla]*

---

## Test 7: `test_fix.py`

**Qué hace:** Ejecuta una verificación básica del sistema en 5 días:
- Verifica que la simulación se ejecute sin errores
- Valida que todos los componentes del resultado estén presentes
- Confirma la presencia de DataFrames clave: `df_productos`, `df_pedidos`, `df_kardex`, `df_compras`, `df_despachos`
- Verifica que `resultados_diarios` y `metricas_globales` estén disponibles
- Reporta el tipo y tamaño de cada componente

**Comando de ejecución:**
```bash
.venv\Scripts\python.exe tests\test_fix.py
```

### Resultado del Test 7:

*[Pegar aquí captura de pantalla]*

---

## Test 8: `verify_committed.py`

**Qué hace:** Ejecuta una simulación de 15 días y realiza validaciones exhaustivas del stock comprometido:
1. **Estado Final:** Muestra el estado de inventario al final de la simulación (Stock Físico, Comprometido, Disponible)
2. **Totales:** Calcula totales de stock físico, comprometido y disponible
3. **Validación de No-Negatividad:** Confirma que el stock físico y comprometido nunca sean negativos
4. **Coherencia:** Verifica que `Stock Disponible = Stock Físico - Stock Comprometido`

**Comando de ejecución:**
```bash
.venv\Scripts\python.exe tests\verify_committed.py
```

### Resultado del Test 8:

*[Pegar aquí captura de pantalla]*

---

## Resumen de Tests

| # | Archivo | Propósito Principal |
|---|---------|--------------------|
| 1 | `test_modules.py` | Validación de imports y clases |
| 2 | `test_stock_logic.py` | Lógica de stock comprometido |
| 3 | `test_compras_kardex.py` | Sistema de compras automáticas |
| 4 | `verify_changes.py` | Verificación integral del sistema |
| 5 | `debug_chart_data.py` | Validación de datos para gráficos |
| 6 | `verify_corrections.py` | Validación de funcionalidades clave |
| 7 | `test_fix.py` | Verificación básica del sistema |
| 8 | `verify_committed.py` | Coherencia de stock comprometido |

**Ejecutar todos los tests:**
```bash
.venv\Scripts\python.exe tests\test_modules.py
.venv\Scripts\python.exe tests\test_stock_logic.py
.venv\Scripts\python.exe tests\test_compras_kardex.py
.venv\Scripts\python.exe tests\verify_changes.py
.venv\Scripts\python.exe tests\debug_chart_data.py
.venv\Scripts\python.exe tests\verify_corrections.py
.venv\Scripts\python.exe tests\test_fix.py
.venv\Scripts\python.exe tests\verify_committed.py
```